##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Ranking Landing Page Example

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/ranking/overview"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/ranking/blob/master/tensorflow_ranking/examples/keras/landingpage_example.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In this landing page example, we build a single tower ranking model using the [MSLR-Web10K dataset](https://www.microsoft.com/en-us/research/project/mslr/) with TF-Ranking.

In [ ]:
# Install TF-Ranking and TF Dataset.
!pip install -q tensorflow-ranking
!pip install -q --upgrade tfds-nightly

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_ranking as tfr

# Load MSLR-WEB10K dataset from TFDS.
ds = tfds.load("mslr_web/10k_fold1", split="train")

# Add a mask tensor.
ds = ds.map(lambda feature_map: {
    "_mask": tf.ones_like(feature_map["label"], dtype=tf.bool),
    **feature_map
})

# Shuffle data and create padded batches for queries with different lengths.
# Padded items will have False on `_mask`.
ds = ds.shuffle(buffer_size=1000).padded_batch(batch_size=32)

# Create (features, labels) tuples from data and set -1 label for masked items.
ds = ds.map(lambda feature_map: (
    feature_map, tf.where(feature_map["_mask"], feature_map.pop("label"), -1.)))

# Create a model with Keras Functional API.
inputs = {
    name: tf.keras.Input(shape=(None, 136), dtype=tf.float32, name=name)
    for name in ds.element_spec[0]
    if name != "_mask"
}
norm_inputs = [tf.keras.layers.BatchNormalization()(x) for x in inputs.values()]
x = tf.concat(norm_inputs, axis=-1)
for layer_width in [128, 64, 32]:
  x = tf.keras.layers.Dense(units=layer_width)(x)
  x = tf.keras.layers.Activation(activation=tf.nn.relu)(x)
scores = tf.squeeze(tf.keras.layers.Dense(units=1)(x), axis=-1)

# Create, compile and train the model.
model = tf.keras.Model(inputs=inputs, outputs=scores)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss=tfr.keras.losses.SoftmaxLoss(),
    metrics=tfr.keras.metrics.get("ndcg", topn=5, name="NDCG@5"))
model.fit(ds, epochs=3)